In [23]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Equilibrage 
from sklearn.utils import resample

# Modele
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

#Export modele
import joblib

In [24]:
df = pd.read_parquet('flights_data.parquet')

In [25]:
# Créer la colonne binaire 'delayed'
df['delayed'] = df['ARR_DELAY'].apply(lambda x: 0 if x < 1 else 1)

In [26]:
df.columns

Index(['MONTH', 'DAY_OF_MONTH', 'UNIQUE_CARRIER', 'FL_NUM', 'ORIGIN_CITY_NAME',
       'ORIGIN_STATE_NM', 'DEST_CITY_NAME', 'CRS_DEP_TIME', 'DISTANCE',
       'ORIGIN_STATE_ABR', 'DEST_STATE_ABR', 'ORIGIN_AIRPORT_ID',
       'DEST_AIRPORT_ID', 'DEP_TIME_BLK', 'ARR_TIME_BLK', 'HISTORICAL_DELAY',
       'ARR_DELAY', 'delayed'],
      dtype='object')

In [27]:
df

,MONTH,DAY_OF_MONTH,UNIQUE_CARRIER,FL_NUM,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,DEST_CITY_NAME,CRS_DEP_TIME,DISTANCE,ORIGIN_STATE_ABR,DEST_STATE_ABR,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_TIME_BLK,ARR_TIME_BLK,HISTORICAL_DELAY,ARR_DELAY,delayed
0,1,6,AA,43,"Dallas/Fort Worth, TX",Texas,"Detroit, MI",1100.0,986.0,TX,MI,11298,11433,1100-1159,1400-1459,24.544169,-6.0,0
1,1,7,AA,43,"Dallas/Fort Worth, TX",Texas,"Detroit, MI",1100.0,986.0,TX,MI,11298,11433,1100-1159,1400-1459,32.000896,-12.0,0
2,1,8,AA,43,"Dallas/Fort Worth, TX",Texas,"Detroit, MI",1100.0,986.0,TX,MI,11298,11433,1100-1159,1400-1459,19.482305,7.0,1
3,1,9,AA,43,"Dallas/Fort Worth, TX",Texas,"Detroit, MI",1100.0,986.0,TX,MI,11298,11433,1100-1159,1400-1459,2.834325,-5.0,0
4,1,10,AA,43,"Dallas/Fort Worth, TX",Texas,"Detroit, MI",1100.0,986.0,TX,MI,11298,11433,1100-1159,1400-1459,21.743780,113.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460944,12,31,WN,1077,"Tucson, AZ",Arizona,"Los Angeles, CA",755.0,451.0,AZ,CA,15376,12892,0700-0759,0800-0859,0.000000,-13.0,0
460945,12,31,WN,1345,"Tucson, AZ",Arizona,"Los Angeles, CA",1320.0,451.0,AZ,CA,15376,12892,1300-1359,1300-1359,41.951208,9.0,1
460946,12,31,WN,1176,"Tucson, AZ",Arizona,"Chicago, IL",705.0,1440.0,AZ,IL,15376,13232,0700-0759,1100-1159,0.000000,-30.0,0
460947,12,31,WN,865,"Tucson, AZ",Arizona,"San Diego, CA",1220.0,368.0,AZ,CA,15376,14679,1200-1259,1200-1259,58.899301,-4.0,0


In [28]:
df = df.drop(columns = ['CRS_DEP_TIME', 'DISTANCE', 'HISTORICAL_DELAY',
                      'ORIGIN_STATE_NM', 'ORIGIN_STATE_ABR', 'DEST_STATE_ABR', 'ORIGIN_AIRPORT_ID',
                      'DEST_AIRPORT_ID', 'ARR_TIME_BLK'])

In [32]:
# Sélection des colonnes à exclure
columns_to_exclude = ['delayed', 'ARR_DELAY']

# Séparation des features (X) et de la cible (y)
X = df.drop(columns=columns_to_exclude)
y = df['delayed']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sélection des caractéristiques numériques et catégorielles
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Pipeline pour les transformations préliminaires
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Pipeline final combinant le prétraitement et le modèle de régression logistique
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Entraîner le pipeline
pipeline.fit(X_train, y_train)

# Prédire avec le modèle entraîné
predictions = pipeline.predict(X_test)

# Calcul des métriques
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
class_report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

MemoryError: Unable to allocate 44.9 GiB for an array with shape (4490009, 1342) and data type float64

In [30]:
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Confusion Matrix:
[[712197  33132]
 [338254  38920]]
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.96      0.79    745329
           1       0.54      0.10      0.17    377174

    accuracy                           0.67   1122503
   macro avg       0.61      0.53      0.48   1122503
weighted avg       0.63      0.67      0.58   1122503



In [31]:
# Chemin vers le fichier de sauvegarde
filename = 'model.joblib'

# Sauvegarder le pipeline
joblib.dump(pipeline, filename)

['model.joblib']